# GeoMap

Matching location data of .las files (LiDAR) with specific tree IDs and ForestGeo Tree Species Labels.

In [14]:
!pip3 install lasio laspy
!pip3 install utm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for utm: filename=utm-0.7.0-py3-none-any.whl size=6108 sha256=d6df21e75a151df365f4e5cd8cae68fd94f6d08f8df07c0442f41c4f746bb13e
  Stored in directory: /root/.cache/pip/wheels/a5/b0/12/7ee4fdb0f9fbb4157100bd02390436ed5d58ebfd3c6d6a0886
Successfully built utm


In [2]:
# Mount Google Drive (where data sit)
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [3]:
# Set Project Folder
import os
header = '/content/drive/My Drive'
hongjin_path = 'classes/2022 fall/CS 288 AI for Social Impact/CS288 Final Project - Tree Species'
derek_path = 'jr/CS288 Final Project - Tree Species'
matt_path = ''

# Select path from above
project_path = os.path.join(header, hongjin_path)
project_path

'/content/drive/My Drive/classes/2022 fall/CS 288 AI for Social Impact/CS288 Final Project - Tree Species'

In [4]:
# Import code utilities files
import sys
sys.path.insert(0, os.path.join(project_path, 'mpala-tree-mapping'))

In [15]:
# import packages
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from osgeo import gdal
import laspy
import utm
import pickle

## Convert location data to lat and long

### ForestGEO Labels

In [6]:
# read data
forestgeo = pd.read_csv(os.path.join(project_path, 'PlotDataReport10-07-2022_1734418034.txt'), delimiter = "\t")
forestgeo.head()

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,No.,Latin,Mnemonic,SubSpecies,Quadrat,PX,PY,TreeID,Tag,StemID,StemTag,Census,DBH,HOM,Date,Codes,Stem,Status
0,1,Acacia brevispica,ACACBR,NaN,221,36.30040,400.50461,124386,20847,254971,020847,1,77.0,0.5,2012-11-20,NaN,main,alive
1,2,Acacia brevispica,ACACBR,NaN,311,51.84520,206.57974,124814,30407,255478,030407,1,37.0,0.5,2012-11-17,NaN,main,alive
2,3,Acacia brevispica,ACACBR,NaN,503,81.23257,58.41180,126361,50086,257294,050086,1,50.0,0.5,2012-11-23,NaN,main,alive
3,4,Acacia brevispica,ACACBR,NaN,10001,1982.57861,9.46110,131025,1000015,262757,1000015,1,23.0,0.5,2014-11-15,M,main,alive
4,5,Acacia brevispica,ACACBR,NaN,10001,1982.57861,9.46110,131025,1000015,262758,1000016,1,23.0,0.5,2014-11-15,NaN,NaN,alive


In [7]:
# convert location data PX and PY to lat and long for data matching
## cite: https://stackoverflow.com/questions/7477003/calculating-new-longitude-latitude-from-old-n-meters

def forestgeo_to_latlong(org_lat, org_long, x_meter, y_meter):
  """
  local helper function
  returns the lat and long of a geo point knowing the base point coordinates in lat and long and distance from the base point in meters
  
  parameters: 
  org_lat: float
    the latitude of the base point
  org_long: float
    the longitude of the base point
  x_meter: float
    the horizontal distance from the base point in meters
  y_meter: float
    the vertical distance from the base point in meters

  returns:
  new_lat: float
    latitude of the new geo point
  new_long: float
    longitude of the new geo point
  """
  r_earth = 6378 # radius of the earth in km
  new_lat = org_lat + (x_meter/1000 / r_earth) * (180 / np.pi)
  new_long = org_long + (y_meter/1000 / r_earth) * (180 / np.pi) / np.cos(org_lat * np.pi / 180)
  return new_lat, new_long

In [8]:
# quick test
import geopy.distance

# base lat long of the mpala plot
# listed on https://forestgeo.si.edu/sites/africa/mpala
base_lat = 0.291800000000
base_long = 36.880900000000

geopy.distance.geodesic((base_lat, base_long), forestgeo_to_latlong(org_lat = base_lat, org_long = base_long, x_meter = 1000, y_meter = 0)).km

0.9933272230594774

In [9]:
# convert location data px and py to lat and long
new_lat_lst = []
new_long_lst = []
for i in range(len(forestgeo)):
  new_lat, new_long = forestgeo_to_latlong(org_lat = base_lat, org_long = base_long, x_meter = forestgeo.iloc[i].PX, y_meter = forestgeo.iloc[i].PY)
  new_lat_lst.append(new_lat)
  new_long_lst.append(new_long)
forestgeo['latitude'] = new_lat_lst
forestgeo['longitude'] = new_long_lst
forestgeo.head()  


,No.,Latin,Mnemonic,SubSpecies,Quadrat,PX,PY,TreeID,Tag,StemID,StemTag,Census,DBH,HOM,Date,Codes,Stem,Status,latitude,longitude
0,1,Acacia brevispica,ACACBR,NaN,221,36.30040,400.50461,124386,20847,254971,020847,1,77.0,0.5,2012-11-20,NaN,main,alive,0.292126,36.884498
1,2,Acacia brevispica,ACACBR,NaN,311,51.84520,206.57974,124814,30407,255478,030407,1,37.0,0.5,2012-11-17,NaN,main,alive,0.292266,36.882756
2,3,Acacia brevispica,ACACBR,NaN,503,81.23257,58.41180,126361,50086,257294,050086,1,50.0,0.5,2012-11-23,NaN,main,alive,0.292530,36.881425
3,4,Acacia brevispica,ACACBR,NaN,10001,1982.57861,9.46110,131025,1000015,262757,1000015,1,23.0,0.5,2014-11-15,M,main,alive,0.309610,36.880985
4,5,Acacia brevispica,ACACBR,NaN,10001,1982.57861,9.46110,131025,1000015,262758,1000016,1,23.0,0.5,2014-11-15,NaN,NaN,alive,0.309610,36.880985


In [12]:
# save new data table for fast processing later
with open(os.path.join(project_path, 'outputs', 'forestgeo_with_latlong.pickle'), 'wb') as f:
    pickle.dump(forestgeo, f, protocol=pickle.HIGHEST_PROTOCOL)

In [13]:
# read saved data table with approximated lat and long
with open(os.path.join(project_path, 'outputs', 'forestgeo_with_latlong.pickle'), 'rb') as f:
    forestgeo = pickle.load(f)
forestgeo.head()

,No.,Latin,Mnemonic,SubSpecies,Quadrat,PX,PY,TreeID,Tag,StemID,StemTag,Census,DBH,HOM,Date,Codes,Stem,Status,latitude,longitude
0,1,Acacia brevispica,ACACBR,NaN,221,36.30040,400.50461,124386,20847,254971,020847,1,77.0,0.5,2012-11-20,NaN,main,alive,0.292126,36.884498
1,2,Acacia brevispica,ACACBR,NaN,311,51.84520,206.57974,124814,30407,255478,030407,1,37.0,0.5,2012-11-17,NaN,main,alive,0.292266,36.882756
2,3,Acacia brevispica,ACACBR,NaN,503,81.23257,58.41180,126361,50086,257294,050086,1,50.0,0.5,2012-11-23,NaN,main,alive,0.292530,36.881425
3,4,Acacia brevispica,ACACBR,NaN,10001,1982.57861,9.46110,131025,1000015,262757,1000015,1,23.0,0.5,2014-11-15,M,main,alive,0.309610,36.880985
4,5,Acacia brevispica,ACACBR,NaN,10001,1982.57861,9.46110,131025,1000015,262758,1000016,1,23.0,0.5,2014-11-15,NaN,NaN,alive,0.309610,36.880985


### LiDAR Images

In [18]:
# Data Paths
lidar_path = os.path.join(project_path, 'MpalaForestGEOPlotData', 'data')
high_res_path = os.path.join(lidar_path, 'HighResAcquisitions')
# segmented tree images
tree_files = os.listdir(os.path.join(lidar_path, 'HighResAcquisitions', 'MpalaForestGEO_LasClippedtoTreePolygons'))
tree_files[:5]

['treeID_42693.las',
 'treeID_42718.las',
 'treeID_42716.las',
 'treeID_42730.las',
 'treeID_42715.las']

In [19]:
sample_tree = laspy.read(os.path.join(high_res_path, 'MpalaForestGEO_LasClippedtoTreePolygons', 'treeID_42693.las'))
list(sample_tree.point_format.dimension_names)

['X',
 'Y',
 'Z',
 'intensity',
 'return_number',
 'number_of_returns',
 'scan_direction_flag',
 'edge_of_flight_line',
 'classification',
 'synthetic',
 'key_point',
 'withheld',
 'scan_angle_rank',
 'user_data',
 'point_source_id',
 'gps_time',
 'red',
 'green',
 'blue',
 'HeightAboveGround']

In [20]:
# convert LiDAR location data to lat long coordinates
## Mpala coordinates: 1.0822850033164138, 37.149144200529584
## cite https://github.com/Turbo87/utm
## UTM zone system: https://www.usgs.gov/faqs/how-are-utm-coordinates-measured-usgs-topographic-maps#:~:text=The%20UTM%20(Universal%20Transverse%20Mercator,Zone%2019%2C%20which%20includes%20Maine.

# get the zone number and zone letter from Mpala's coordinates
utm.from_latlon(1.0822850033164138, 37.149144200529584)

(294046.54747902136, 119688.00302557605, 37, 'N')

In [32]:
# sample tree point lat longs 
scaleX = sample_tree.header.scale[0]
scaleY = sample_tree.header.scale[1]
sample_latlong = utm.to_latlon(sample_tree.X*scaleX, sample_tree.Y*scaleY, 37, 'N')
print(sample_latlong)
# approximate the location of the tree by taking the average of the points 
print(np.average(sample_latlong, axis = 1))

(array([0.28432283, 0.28432481, 0.28433702, ..., 0.2843334 , 0.28433394,
       0.28433376]), array([36.86906305, 36.86906296, 36.86907876, ..., 36.86904401,
       36.86904563, 36.86904518]))
[ 0.28433333 36.86906439]


In [57]:
%timeit
# generate the lat and long of points for each tree and save outputs for downstream matching

latlong_dict = {}
for t in tree_files:
  # read las file 
  las = laspy.read(os.path.join(high_res_path, 'MpalaForestGEO_LasClippedtoTreePolygons', t))
  scaleX = las.header.scale[0]
  scaleY = las.header.scale[1]
  latlongs = utm.to_latlon(las.X*scaleX, las.Y*scaleY, 37, 'N')
  latlongs_avg = np.average(latlongs, axis = 1)
  latlong_dict[t[:-4]] = latlongs, latlongs_avg

# save new data table for fast processing later
with open(os.path.join(project_path, 'outputs', 'lidar_with_latlong.pickle'), 'wb') as f:
    pickle.dump(latlong_dict, f, protocol=pickle.HIGHEST_PROTOCOL)

KeyboardInterrupt: ignored

^ stopped early because it is taking a long time. We should verify a few samples before conversion with the full dataset.

In [59]:
len(latlong_dict)

9473

In [60]:
# save new data table for fast processing later
with open(os.path.join(project_path, 'outputs', 'lidar_with_latlong.pickle'), 'wb') as f:
    pickle.dump(latlong_dict, f, protocol=pickle.HIGHEST_PROTOCOL)

# read saved data table with approximated lat and long
with open(os.path.join(project_path, 'outputs', 'lidar_with_latlong.pickle'), 'rb') as f:
    latlong_dict = pickle.load(f)

In [62]:
lat_lst = []
long_lst = []
for tree_id in latlong_dict.keys():
  lat_lst.append(latlong_dict[tree_id][1][0])
  long_lst.append(latlong_dict[tree_id][1][1])

tree_dict = pd.DataFrame()
tree_dict['tree_id'] = latlong_dict.keys()
tree_dict['latitude'] = lat_lst
tree_dict['longitude'] = long_lst
tree_dict.head()

,tree_id,latitude,longitude
0,treeID_42693,0.284333,36.869064
1,treeID_42718,0.284321,36.871805
2,treeID_42716,0.284316,36.870929
3,treeID_42730,0.284316,36.871608
4,treeID_42715,0.284332,36.870756


In [63]:
len(tree_dict)

9473

## Match species labels with LiDAR images

In [65]:
forestgeo.head()

,No.,Latin,Mnemonic,SubSpecies,Quadrat,PX,PY,TreeID,Tag,StemID,StemTag,Census,DBH,HOM,Date,Codes,Stem,Status,latitude,longitude
0,1,Acacia brevispica,ACACBR,NaN,221,36.30040,400.50461,124386,20847,254971,020847,1,77.0,0.5,2012-11-20,NaN,main,alive,0.292126,36.884498
1,2,Acacia brevispica,ACACBR,NaN,311,51.84520,206.57974,124814,30407,255478,030407,1,37.0,0.5,2012-11-17,NaN,main,alive,0.292266,36.882756
2,3,Acacia brevispica,ACACBR,NaN,503,81.23257,58.41180,126361,50086,257294,050086,1,50.0,0.5,2012-11-23,NaN,main,alive,0.292530,36.881425
3,4,Acacia brevispica,ACACBR,NaN,10001,1982.57861,9.46110,131025,1000015,262757,1000015,1,23.0,0.5,2014-11-15,M,main,alive,0.309610,36.880985
4,5,Acacia brevispica,ACACBR,NaN,10001,1982.57861,9.46110,131025,1000015,262758,1000016,1,23.0,0.5,2014-11-15,NaN,NaN,alive,0.309610,36.880985


In [66]:
tree_dict.head()

,tree_id,latitude,longitude
0,treeID_42693,0.284333,36.869064
1,treeID_42718,0.284321,36.871805
2,treeID_42716,0.284316,36.870929
3,treeID_42730,0.284316,36.871608
4,treeID_42715,0.284332,36.870756


In [79]:
# initial matching attempt
match_digit = 3
tree_dict['latitude_short'] = tree_dict['latitude'].round(match_digit)
tree_dict['longitude_short'] = tree_dict['longitude'].round(match_digit)
forestgeo['latitude_short'] = forestgeo['latitude'].round(match_digit)
forestgeo['longitude_short'] = forestgeo['longitude'].round(match_digit)

pd.merge(forestgeo[forestgeo['Stem'] == 'main'][['TreeID', 'Latin', 'latitude_short', 'longitude_short']], tree_dict[['tree_id', 'latitude_short', 'longitude_short']], how='inner', on = ['latitude_short', 'longitude_short'])

,TreeID,Latin,latitude_short,longitude_short,tree_id
